# Stage 1: Scan Simulation/Rendering

## Initialization and imports

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("/home/sebastian/Projects/SRB/Package/repo/scanner-sim")

In [11]:
from simulator.rendering import *

data_path = "../data/test"
config, cam_geom = {}, load_calibration("../scanner/calibration/camera/camera_geometry.json")
# Render small crop (200 pixels high) only for optimal performance
cam_geom["image_width, pixels"] = 100
cam_geom["image_height, pixels"] = 100

configure_camera_geometry(config, cam_geom)
configure_camera_focus(config, "../scanner/calibration/camera/camera_focus.json")

configure_projector_geometry(config, "../scanner/calibration/projector/projector_geometry.json")
configure_projector_focus(config, "../scanner/calibration/projector/projector_focus.json")
config["proj_diffLimit"] = config["cam_diffLimit"] * config["cam_aperture"] / config["proj_aperture"]
# config["proj_diffLimit"] = 0
# config["proj_aperture"] = 0.001


print("Config:", config)

header, body = load_template("projector_focus.xml")
ensure_exists(data_path + "/")

h, w = 1080, 1920
pattern = np.zeros((h, w, 3), dtype=np.uint8)
# pattern[:, :, 0] = np.random.randint(255, size=(h, w))
pattern[h-1, w//2, :] = 255
# pattern[:, w//2 - 10, :] = 255
# pattern[:, w//2 - 8, :] = 255
# pattern[h-10, :, :] = 255
# pattern[h-8, :, :] = 255
imageio.imwrite(data_path + "/pattern.png", pattern)
# imageio.imwrite(data_path + "/pattern.png", pattern[::10, ::10, :])

config["scale"] = scale
config["offset"] = config["cam_focus"] - config["proj_focus"]
config["proj_offsetX"], config["proj_offsetY"] = w/2, h

for dist_cm in range(*range_cm):
    # Geometry cube is 2*scale meters in size
    config["dist"] = config["scale"] + dist_cm / 100.
    generate_scene(header, body, config, data_path + "/dist_%d.xml" % dist_cm)

source(mitsuba_path + "/setpath.sh")
render_scenes(data_path + "/dist_*.xml", verbose=verbose)

Config: {'cam_width': 100, 'cam_height': 100, 'cam_fov_y': 0.38946283611979604, 'cam_pixelAspect': 0.9992771751028543, 'cam_focus': 0.8103669534556511, 'cam_aperture': 0.002, 'cam_diffLimit': 0.01028416270425193, 'proj_width': 1920, 'proj_height': 1080, 'proj_scaleX': 2805.329833984375, 'proj_scaleY': 2805.11572265625, 'proj_offsetX': 958.3337814131664, 'proj_offsetY': 1095.1464166282676, 'proj_intensity': 10, 'proj_gapSize': 0, 'proj_focus': 0.4902099376513703, 'proj_aperture': 0.0035951384447263915, 'proj_diffLimit': 0.0057211497483984144}


FileNotFoundError: [Errno 2] No such file or directory: 'projector_focus.xml'

In [ ]:
import os
import json
from IPython.display import Image
import matplotlib.pyplot as plt

#os.chdir("/sls/scanner-sim")
from rendering import render_object, generate_render_parameters

def save_config_and_run_scan(config_name, config):
    # Store rendering parameters for reference
    with open(config_name, "w") as fi:
        json.dump(config, fi, indent=2, sort_keys=True)  
    
    # Run render job which takes roughly 10 minutes to finish all images of the scan
    render_object(config_name)

## Rendering an object with default Gray code patterns

As a first step, we need to define the parameters for the simulation/renderer. This is done with the *generate_render_parameters()* function which supports many different settings. In this example, let's choose the shapes object.

In [ ]:
pars = generate_render_parameters(obj_path="objects/machined/shapes.obj", # Path to the object (relative to the data folder)
                                  result_path="results/shapes", # Path where the results are stored
                                  typ="shapes", # Predefined settings for the shapes object to match the real scanner
                                  size="small", # Resolution of the renderings (large=original camera size, medium=original/2, small=original/4)
                                  samples=64, # Samples per pixel that are used for the rendering
                                  patterns="patterns/gray" # The patterns that are used for the scanning process
                                 ) 

This generates a parameter dictionary with default settings (matching the physical scanner) which can be subsequently modified if needed. The parameter dictionary is then stored and the simulation/rendering is started with the following code:

In [ ]:
save_config_and_run_scan("data/configs/parameters_shapes.json", pars)

After rendering the images, let's look at some results...

In [ ]:
Image(filename="data/results/shapes/img_016.png") 

In [ ]:
Image(filename="data/results/shapes/img_040.png") 

## Rendering an object with different light coding patterns

In order to use different patterns, the path to the patterns and additional settings can be modified. The following example shows how to scan the pawn object with micro phase shifting patterns.

In [ ]:
pars = generate_render_parameters(obj_path="objects/machined/pawn.obj", # The pawn object
                                  result_path="results/pawn", 
                                  typ="pawn",
                                  size="small",
                                  samples=64,
                                  cpu_count=8, # Render with only 8 cores this time
                                  patterns="patterns/mps/32-08", # Select micro phase shifting patterns
                                 )

pars["pattern_colored"] = False # We don't need support for colored patterns this time
pars["pattern_calibrate"] = True # Predistort the patterns so that they match the physical projector
pars["pattern_flip_ud"] = True # Flip the patterns vertically so that they are projected with an upright projector

save_config_and_run_scan("data/configs/parameters_pawn.json", pars)

In [ ]:
Image(filename="data/results/pawn/img_000.png") 

## Rendering a colored/textured object (one single scan)

Textured objects can be scanned/rendered similarly. Currently, only objects in *obj* format are supported. In this example, scan the object from only one direction.

In [ ]:
pars = generate_render_parameters(obj_path="objects/colored/vase/vase.obj", 
                                  result_path="results/vase", 
                                  typ="vase", # Render with vase presets
                                  size="small",
                                  samples=64,
                                  cpu_count=8,
                                  patterns="patterns/gray"
                                 )

pars["pattern_colored"] = True

save_config_and_run_scan("data/configs/parameters_vase.json", pars)

Visualize the results again...

In [ ]:
Image(filename="data/results/vase/img_018.png") 

## Rendering a colored/textured object (multiple scans, turntable)

In this example, we scan the object from multiple directions to reproduce the full geometry. The object is rotated on a virtual turntable and all scans can be merged together in the reconstruction phase (see below). Note that this rendering process will take a while (~20 minutes).

In [ ]:
pars = generate_render_parameters(obj_path="objects/colored/vase/vase.obj", 
                                  result_path="results/vase_full", 
                                  typ="vase", # Render with vase presets
                                  size="small",
                                  samples=32, # Let's reduce the samples for faster scanning
                                  cpu_count=8,
                                  patterns="patterns/gray"
                                 )

pars["pattern_colored"] = True
pars["rot_type"] = "Turntable" # Enable turntable rotation
pars["rot_range"] = [0, 360, 4] # Rotate between 0 and 360 degrees in 4 steps

save_config_and_run_scan("data/configs/parameters_vase_full.json", pars)

Let's see the 4 different views...

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(20,15))
for i in range(4):
    img = plt.imread("data/results/vase_full/rot_%03i/img_000.png"%i)
    axs[i].imshow(img)
    axs[i].axis('off')

# Stage 2: Decoding and Reconstruction

Attention: This stage depends on the results that were produced in stage 1. If you haven't run the relevant cells above, some cells will not work.

## Initialization and imports

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import glob
import json
from tqdm.auto import tqdm

#os.chdir("/sls/scanner-sim")
import meshplot as mp
from decoding import decode_gray, decode_mps
from reconstruction import reconstruct_single, merge_single
from utils import load_projector_calibration, load_camera_calibration, numpinize, load_openexr

## Decoding of a single scan (shapes object with Gray code)

In [ ]:
data_path = "data/results/shapes"

# Decode Gray coded images
idx_h, idx_v = decode_gray(data_path, group=True, plot=True, sim=True, threshold=130)

## Reconstruction of a single scan (shapes object with Gray code)

In this example, we reconstruct the depth map and the colored point cloud from the correspondence indices that were created during the decoding stage. The indices from the decoding stage don't need to be passed into the function, they are passed as files with configured filenames. In addition, we need to load the projector and camera calibration for the reconstruction.

In [ ]:
# Load projector and camera calibration
pro_calib = load_projector_calibration("data/calibrations/projector_calibration_new.json")[2]
cam_calib = load_camera_calibration("data/calibrations/camera_calibration_s_new.json")

data_path = "data/results/shapes"

points, colors, depth_map = reconstruct_single(data_path, cam_calib, pro_calib, plot=True, sim=True)

How does the reconstructed depthmap compare to the ground truth depth map? For this we load the ground truth depth map from the renderings and plot the difference and a histogram over the difference.

In [ ]:
img, gt_depth_map = load_openexr("data/results/shapes/img_000.exr", load_depth=True)
difference = gt_depth_map - depth_map
hist_difference = np.abs(difference.reshape(-1))

fig, axs = plt.subplots(1, 2, figsize=(20,7))
axs[0].imshow(difference)
axs[0].axis('off')
axs[1].hist(difference.reshape(-1), bins=np.linspace(0.00001, 0.002, 40))
plt.show()

Finally, let us also visualize the reconstructed point cloud...

In [ ]:
mp.plot(points, c=colors, shading={"point_size": 5.0});

## Decoding and reconstruction of multiple scans (vase object with Gray code)

Similar to before, we can decode the multiple scans of the colored vase object.

In [ ]:
# Load projector and camera calibration
pro_calib = load_projector_calibration("data/calibrations/projector_calibration_new.json")[2]
cam_calib = load_camera_calibration("data/calibrations/camera_calibration_s_new.json")

def decode_and_reconstruct(data_path):
    decode_gray(data_path, group=True, sim=True, threshold=50)
    reconstruct_single(data_path, cam_calib, pro_calib, sim=True)


data_paths = glob.glob("data/results/vase_full/rot_*")

for data_path in tqdm(data_paths):
    decode_and_reconstruct(data_path)

Now merge all the decoded and reconstructed scans together by rotating them to the same coordinate system.

In [ ]:
# Load the turntable calib
stage_calib = numpinize(json.load(open("data/calibrations/stage_calibration_sim.json", "r")))

# Merge the single scans
points, normals, colors = merge_single("data/results/vase_full", "/rot_%s/reconstructed/group_points.ply"%"%03i",
                                     stage_calib, title="vase.ply", max_range=4, sim=True)
    
# Plot the reconstructed point cloud
mp.plot(points, c=colors, shading={"point_size": 5.0});

## Decoding of different patterns (pawn with micro phase shifting)

It is possible to use different patterns for the scanning proces. This can be achieved by adding the patterns as images and a decoding stage, which reads in the rendered results and produces the correspondence map. For the three decoding strategies listed in the paper, we supply an implementation with our software. The following example shows the decoding for micro phase shifting patterns.

In [ ]:
ic_h = decode_mps("data/results/pawn", "data/patterns/mps/32-08/", cam=[1616, 1213])

plt.figure(figsize=(20, 10))
plt.imshow(ic_h)
plt.axis('off')
plt.show()